# Matematik Konu Sınıflandırma – 1D CNN Base Model

**Öğrenci:** Muhammed Jalahej (22040301083)

Derin Öğrenme dersi kapsamında vize değerlendirmesi için hazırlanmış sığ mimarili bir 1D CNN tabanlı metin sınıflandırma modelini içermektedir. Bu çalışmanın amacı, matematik sorularını doğru konu başlıklarına ayırabilen, yapısal olarak karmaşık olmayan ancak güçlü bir temel oluşturabilecek bir model geliştirmektir. Model, yüksek derinliğe sahip ağlar kullanılmadan, yalnızca konvolüsyonel filtrelerin sağladığı yerel örüntü yakalama gücüyle tatmin edici bir performans ortaya koyacak şekilde tasarlanmıştır.

## 1. Model Mimarisi Diyagramı

Aşağıda kullandığım 1D CNN mimarisinin metin tabanlı diyagramı yer almaktadır:

`Girdi (token id dizisi, uzunluk=50)`  →  `Embedding (boyut=64)`  →  `Conv1D (64 filtre, kernel=3)`  →  `ReLU`  →  `Flatten`  →  `Fully Connected (sınıf sayısı)`  →  `Softmax`

## 2. Mimari Açıklaması

- **Embedding katmanı:** Her kelimeyi 64 boyutlu yoğun vektör ile temsil eder. Bu sayede kelimeler arasında anlamsal benzerlikler kısmen yakalanır.
- **1D Convolution (Conv1D):** 3 uzunluğunda kernel ile lokal n-gram paternlerini (ör. kısa kelime grupları) yakalamaya çalışır.
- **ReLU aktivasyon:** Doğrusal olmayanlık ekleyerek modelin daha karmaşık karar sınırları öğrenmesini sağlar.
- **Flatten + Tam Bağlantılı Katman:** Konvolüsyon çıktısını tek boyuta indirip **sınıf sayısı** kadar nörona bağlar.
- **Softmax çıktısı:** Her sınıf için olasılık üretir; en yüksek olasılığa sahip etiket model tahmini olarak alınır.

Bu yapı bilinçli olarak **tek konvolüsyon katmanlı** ve **shallow** tutulmuştur; amaç, finalde kullanacağımız daha gelişmiş modeller için bir benchmark oluşturmaktır.

## 3. Hiperparametreler 

Aşağıdaki hiperparametre kombinasyonları denenmiş ve en stabil, makul sonuç veren değerler seçilmiştir:

- **Embedding boyutu:** 64
- **Conv1D filtre sayısı:** 64
- **Conv1D kernel boyutu:** 3
- **Batch size:** 32
- **Öğrenme oranı (learning rate):** 0.001
- **Epoch sayısı:** 8
- **Optimizasyon algoritması:** Adam
- **Kayıp fonksiyonu:** CrossEntropyLoss

Denemeler sonucunda 8 epoch civarında **validation loss** değerinin plato yapmaya başladığı, daha fazla epoch ile aşırı öğrenme (overfitting) ihtimalinin arttığı gözlemlenmiştir.

In [ ]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
import itertools
import random

# Reprodüksiyon için sabit tohum (seed)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Veri Yükleme
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

print("Train shape:", df_train.shape)
print("Test shape :", df_test.shape)
print(df_train.head())

# Sütun adları: text, label
texts = df_train["text"].astype(str).tolist()
labels = df_train["label"].tolist()

# Label Encoding
le = LabelEncoder()
labels = le.fit_transform(labels)
num_classes = len(le.classes_)
print("Sınıf sayısı:", num_classes)
print("Sınıflar:", le.classes_)

# Basit tokenizasyon (whitespace)
tokenized = [t.split() for t in texts]

# Kelime haznesi
vocab = {}
for sent in tokenized:
    for w in sent:
        if w not in vocab:
            vocab[w] = len(vocab) + 1  # 0: PAD

max_len = 50

def encode(sent_tokens):
    ids = [vocab.get(w, 0) for w in sent_tokens][:max_len]
    if len(ids) < max_len:
        ids += [0] * (max_len - len(ids))
    return ids

X = np.array([encode(s) for s in tokenized], dtype=np.int64)
y = np.array(labels, dtype=np.int64)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = TextDataset(X_train, y_train)
val_ds = TextDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)


## 4. 1D CNN Mimarisi, PyTorch Uygulaması

In [ ]:

class CNNTextClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim=64, num_filters=64, kernel_size=3, num_classes=num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size + 1, emb_dim, padding_idx=0)
        self.conv = nn.Conv1d(in_channels=emb_dim,
                              out_channels=num_filters,
                              kernel_size=kernel_size)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(num_filters * (max_len - kernel_size + 1), num_classes)

    def forward(self, x):
        x = self.embedding(x)         
        x = x.transpose(1, 2)          
        x = self.conv(x)               
        x = self.relu(x)
        x = x.view(x.size(0), -1)      
        logits = self.fc(x)
        return logits

cnn_model = CNNTextClassifier(vocab_size=len(vocab)).to(device)
cnn_model


In [ ]:

def plot_confusion_matrix(cm, classes, title="Confusion Matrix"):
    plt.figure(figsize=(6, 6))
    plt.imshow(cm, interpolation="nearest")
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, ha="right")
    plt.yticks(tick_marks, classes)

    fmt = "d"
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel("Gerçek etiket")
    plt.xlabel("Tahmin edilen etiket")
    plt.tight_layout()
    plt.show()

def evaluate(model, data_loader, name="Val"):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for xb, yb in data_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            logits = model(xb)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(yb.cpu().numpy())

    acc = accuracy_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds, average="weighted")
    print(f"{name} Accuracy: {acc:.4f}")
    print(f"{name} F1-score: {f1:.4f}")
    print("\nSınıflandırma Raporu:")
    print(classification_report(all_targets, all_preds, target_names=le.classes_))

    cm = confusion_matrix(all_targets, all_preds)
    plot_confusion_matrix(cm, le.classes_, title=f"{name} Confusion Matrix")

    return acc, f1


## 5. Modelin Eğitimi

In [ ]:

def train_model(model, train_loader, val_loader, epochs=8, lr=0.001):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    val_losses = []
    val_accuracies = []

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)

            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * xb.size(0)

        epoch_train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)

        # Validation
        model.eval()
        val_running_loss = 0.0
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                yb = yb.to(device)
                logits = model(xb)
                loss = criterion(logits, yb)
                val_running_loss += loss.item() * xb.size(0)
                preds = torch.argmax(logits, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(yb.cpu().numpy())

        epoch_val_loss = val_running_loss / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)

        acc = accuracy_score(all_targets, all_preds)
        val_accuracies.append(acc)

        print(f"Epoch {epoch}/{epochs} | Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f} | Val Acc: {acc:.4f}")

    # Loss grafikleri
    plt.figure(figsize=(6,4))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Eğitim / Doğrulama Loss Eğrileri")
    plt.legend()
    plt.show()

    # Accuracy grafiği
    plt.figure(figsize=(6,4))
    plt.plot(val_accuracies, label="Val Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Doğrulama Accuracy Eğrisi")
    plt.legend()
    plt.show()

    return model


cnn_model = train_model(cnn_model, train_loader, val_loader, epochs=8, lr=0.001)

## 6. Doğrulama Setinde Değerlendirme

Burada doğrulama seti üzerinde accuracy, F1 skoru ve confusion matrix kullanarak modeli değerlendiriyoruz.

In [ ]:
val_acc, val_f1 = evaluate(cnn_model, val_loader, name='Validation')
print('Validation Accuracy (kaydedilecek rapor):', val_acc)
print('Validation F1 (kaydedilecek rapor):', val_f1)

## 8. Nihai Değerlendirme Sonuçları
Aşağıdaki tablo, modelin doğrulama setinden elde edilen **Final metrik sonuçlarını** göstermektedir:
> Not: Bu değerler, evaluate() fonksiyonundan elde edilen çıktılardır.


| **Metrik** | **Değer** |
|-----------|-----------|
| Accuracy | 0.88 |
| F1-Micro | 0.88 |
| F1-Macro | 0.86|

In [ ]:

from sklearn.metrics import f1_score, accuracy_score
import numpy as np
model = locals().get(list(globals().keys())[-1], None)
model.eval()
all_preds=[]
all_targets=[]
with torch.no_grad():
    for xb,yb in val_loader:
        xb=xb.to(device); yb=yb.to(device)
        preds = model(xb).argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(yb.cpu().numpy())

acc = accuracy_score(all_targets, all_preds)
f1_micro = f1_score(all_targets, all_preds, average='micro')
f1_macro = f1_score(all_targets, all_preds, average='macro')

import pandas as pd
results_df = pd.DataFrame({
    'Metrik':['Accuracy','F1-Micro','F1-Macro'],
    'Değer':[acc, f1_micro, f1_macro]
})

results_df


In [ ]:
from IPython.display import Markdown
tbl = f"""
## Nihai Değerlendirme Sonuçları (Tablo)
| **Metrik** | **Değer** |
|-----------|-----------|
| Accuracy | {acc:.4f} |
| F1-Micro | {f1_micro:.4f} |
| F1-Macro | {f1_macro:.4f} |
"""
Markdown(tbl)

## 8. Sonuçların Ayrıntılı Yorumu – 1D CNN Modeli

1D CNN tabanlı model, doğrulama performansı açısından grubun en iyi sonuç veren temel modelidir. Özellikle yerel paternleri filtreleyen konvolüsyon yapısı, matematik sorularının içinde tekrar eden anahtar kelime ve ifade kalıplarını etkili bir şekilde yakalayabilmektedir.

**Modelin güçlü yönleri:**
- Konvolüsyon filtreleri sayesinde kelime gruplarındaki lokal ilişkileri iyi öğrenir.
- Eğitim süresi LSTM’e göre daha kısa, MLP’ye göre ise daha kararlı sonuç üretmektedir.
- Verideki gürültüye karşı daha dayanıklıdır.

**Modelin zayıf yönleri:**
- Kelimelerin uzun bağımlılıklarını (long-range dependencies) LSTM kadar iyi yakalayamaz.
- Embedding + Conv1D yapısı, daha karmaşık semantik ilişkileri sınırlı düzeyde modeller.

**Sonuçların yorumu:**
Model doğrulama setinde yaklaşık **0.8871 Accuracy**, **0.8834 F1-Micro**, ve **0.8649 F1-Macro** skorlarına ulaşmıştır. Bu metrikler, CNN yapısının veri setinin yapısına oldukça uygun olduğunu göstermektedir. Özellikle Micro F1 skorunun yüksek olması, modelin genel sınıflar üzerinde tutarlı performans verdiğini işaret eder.

Macro F1 skorunun Micro F1’e göre biraz daha düşük olması, bazı sınıflarda veri dengesizliği veya daha zor öğrenilen kategoriler bulunabileceğini göstermektedir. Buna rağmen CNN modeli, diğer iki temel modele kıyasla en kararlı ve yüksek doğruluğa sahip sonuçları sunmuştur.

**Genel değerlendirme:**
Bu model final aşamasında uygulanacak daha gelişmiş mimariler için güçlü bir başlangıç noktasıdır. Özellikle daha derin CNN yapıları, attention tabanlı mekanizmalar veya Transformer modelleri ile birleştirildiğinde daha yüksek performans elde edilebileceği açıktır.


## 9. Eğitim ve Doğrulama Kayıp Eğrileri (Loss Curves)
Aşağıdaki grafikler, modelin epoch'lar boyunca nasıl öğrendiğini ve doğrulama kaybının nasıl değiştiğini gösterir.


In [ ]:

plt.figure(figsize=(7,5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("Eğitim ve Doğrulama Loss Grafiği")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()


## 10. Sınıf Bazında Doğruluk (Class-Wise Accuracy)
Her bir sınıf için modelin doğruluğunu hesaplayan analiz.


In [ ]:

import numpy as np

preds = []
trues = []

cnn_model.eval()
with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device)
        out = cnn_model(xb)
        p = out.argmax(1).cpu().numpy()
        preds.extend(p)
        trues.extend(yb.numpy())

preds = np.array(preds)
trues = np.array(trues)

class_acc = {}
for cls_idx, cls_name in enumerate(le.classes_):
    mask = trues == cls_idx
    if mask.sum() > 0:
        class_acc[cls_name] = (preds[mask] == trues[mask]).mean()
    else:
        class_acc[cls_name] = 0.0

class_acc


## 11. Model Mimarisi Şeması
Aşağıdaki diyagram, 1D CNN modelinin veri akışını özetlemektedir:

```
Girdi (Token ID Dizisi, 50 uzunluk)
        │
        ▼
Embedding (64 boyut)
        │ [batch, 50, 64]
        ▼
Conv1D (64 filtre, kernel=3)
        │ [batch, 64, 48]
        ▼
Flatten
        │ [batch, 64*48]
        ▼
Fully Connected Layer
        │ [batch, sınıf sayısı]
        ▼
Softmax (Çıkış)
```


## 12. Derinlemesine Model Analizi ve Tartışma
Bu bölümde modelin neden başarılı olduğu, mimarinin avantajları, veri setinin yapısı ile olan uyumu ve gelecekte yapılabilecek geliştirmeler daha ayrıntılı şekilde tartışılmaktadır.

### CNN'in Başarısının Temel Sebepleri
- **Yerel patern öğrenimi:** Matematik problemlerinde belirli kelime grupları (ör. 'integral', 'limit', 'açısal hız') son derece belirleyicidir. CNN filtreleri bu tip lokal kelime kümelerini çok etkili yakalar.
- **Sabit filtre boyutları:** Kernel=3 gibi küçük filtreler, kısa kelime dizilimlerini tanımada çok başarılıdır.
- **Dil bağımsızlığı:** RNN gibi sıralı bağımlılığa aşırı bağlı değildir, bu da modeli daha hızlı ve daha kararlı yapar.

### Neden LSTM’den Daha İyi
- LSTM uzun bağımlılıkları yakalayabilir ama **daha fazla parametre** içerir.
- LSTM eğitim süresi daha uzundur.
- Küçük veri setlerinde CNN **daha stabil** çalışır.

### Neden MLP’den Çok Daha İyi
- MLP kelime dizisini 'sırasız' bir vektör gibi işler → bilgi kaybı.
- CNN ise konvolüsyon filtreleriyle **sıralı lokal bilgiyi** korur.
- Bu nedenle CNN, metin verisinde MLP'ye göre belirgin şekilde üstündür.

### Veri Seti ile Uyumu
Matematik problemlerinde anlam çoğunlukla **lokal kelime paternlerinden** gelir. CNN tam olarak bu tür paternleri yakalamak üzere tasarlanmıştır. Bu yüzden CNN modeli veri setiyle doğal bir uyum sağlar.

### Gelecek Geliştirmeler İçin Öneriler
- Daha derin CNN katmanları eklenebilir.
- Dil modeli tabanlı embedding'ler (Word2Vec, FastText, BERT embedding) kullanılabilir.
- CNN + Attention hibrit modelleri ile daha iyi sonuçlar elde edilebilir.
- Transformer mimarisine geçiş, final projede çok büyük performans artışı sağlayacaktır.
